# 네이버 뉴스 데이터 가져오기

> 1. 네이버 오픈 API로 기사 URL 크롤링
> 2. bs4 + selenium으로 기사 제목 및 내용 크롤링

In [1]:
import os
import sys
import urllib.request
import requests

news_data = []
page_count = 3

client_id = "NtncpyepC1ObMGo66Pni"
client_secret = "yH0BHrLAED" # 발급받은 client_secret
encText = urllib.parse.quote("파이썬")

for idx in range(page_count):
    # json 결과
    url = "https://openapi.naver.com/v1/search/news?query=" + encText + "&start=" + str(idx * 10 + 1)
    # url = "https://openapi.naver.com/v1/search/blog.xml?query=" + encText # xml 결과
    request = urllib.request.Request(url)
    request.add_header("X-Naver-Client-Id",client_id)
    request.add_header("X-Naver-Client-Secret",client_secret)
    response = urllib.request.urlopen(request)
    rescode = response.getcode()

    if(rescode==200):
    #    response_body = response.read()
        result = requests.get(response.geturl(),
                              headers={"X-Naver-Client-Id":client_id,
                                       "X-Naver-Client-Secret":client_secret}
                             )
        news_data.append(result.json())
    #    print(response_body.decode('utf-8'))
    else:
        print("Error Code:" + rescode)

In [2]:
#print(news_data)
print(len(news_data))
#print(news_data[0])
#print(news_data[0]['items'])
#print(len(news_data[0]['items']))
#print(news_data[0]['items'][2])
#print(news_data[0]['items'][7]['link'])

3


In [3]:
naver_news_link = []

for page in news_data:
    #print(page)
    page_news_link = []
    
    for item in page['items']:
        #print(item)
        temp_link = item['link']
        #print(temp_link)
        if "naver" in temp_link:
            page_news_link.append(temp_link)
    
    naver_news_link.append(page_news_link)
        

# 사이트 확인하기에 편한 코드 구조.
for page in naver_news_link:
    for link in page:
        print(link)

https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=105&oid=018&aid=0005099858
https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=101&oid=023&aid=0003657227
https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=102&oid=003&aid=0010867309
https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=101&oid=030&aid=0002985207
https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=105&oid=029&aid=0002708826
https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=105&oid=092&aid=0002240720


In [4]:
import pandas as pd
import numpy as np
from selenium import webdriver
from tqdm import tqdm_notebook
import requests
import pickle
import re
import ast

from bs4 import BeautifulSoup 
from urllib.request import urlopen
import urllib
import time

In [6]:
# 가상 크롬드라이버를 불러옴.
# 윈도우 10의 경우 chromedriver.exe
driver = webdriver.Chrome('/Users/leesh970930/Desktop/chromedriver')

In [7]:
naver_news_title = []
naver_news_content = []


for n in tqdm_notebook(range(len(naver_news_link))):
    #print(n)
    news_page_title = []
    news_page_content = []
    
    for idx in tqdm_notebook(range(len(naver_news_link[n]))):
        
        
    ########### 긁어온 URL로 접속하기 ############    
        try:
            driver.get(naver_news_link[n][idx])
            print(naver_news_link[n][idx])
            
        except:
            print("Timeout!")
            continue
        
        
        try:
            response = driver.page_source
            
        except UnexpectedAlertPresentException:
            driver.switch_to_alert().accept()
            print("게시글이 삭제된 경우입니다.")
            continue
        
        soup = BeautifulSoup(response, "html.parser")
        
        ###### 뉴스 타이틀 긁어오기 ######
        
        title = None
        
        try:
            item = soup.find('div', class_="article_info")
            title = item.find('h3', class_="tts_head").get_text()
            #print(title)

        except:
            title = "OUTLINK"
        
        #print(title)
        news_page_title.append(title)
        
        
        ###### 뉴스 본문 긁어오기 ######
        
        doc = None
        text = ""
                
        data = soup.find_all("div", {"class" : "_article_body_contents"})
        if data:
            for item in data:

                text = text + str(item.find_all(text=True)).strip()
                text = ast.literal_eval(text)
                doc = ' '.join(text)
   
        else:
            doc = "OUTLINK"
            
        news_page_content.append(doc.replace('\n', ' '))

                
    naver_news_title.append(news_page_title)
    naver_news_content.append(news_page_content)

    time.sleep(2)
    
    
print(naver_news_title[0])
print("==================================")
print(naver_news_content[0])

<ipython-input-7-326f55771db9>:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for n in tqdm_notebook(range(len(naver_news_link))):


<ipython-input-7-326f55771db9>:10: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for idx in tqdm_notebook(range(len(naver_news_link[n]))):


https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=105&oid=018&aid=0005099858
https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=101&oid=023&aid=0003657227
https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=102&oid=003&aid=0010867309



https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=101&oid=030&aid=0002985207



https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=105&oid=029&aid=0002708826
https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=105&oid=092&aid=0002240720


["메타, 'AI 연구개발' 위해 아마존 클라우드 더 쓴다", '[스타트업] 박지웅과 ‘커리의 3점슛 이론’', '[교육소식]배재대 IPP사업단, 일학습병행사업 IT기업 설명회 등']
["   본문 내용     TV플레이어     // TV플레이어     // flash 오류를 우회하기 위한 함수 추가 function _flash_removeCallback() {}    전략적 클라우드 공급자 선정, 장기 계약 자체 데이터센터 보완, AI 연구개발 가속 서드파티와 협업에도 활용 AWS서 실행되는 딥러닝 프레임워크 '파이토치' 성능 개선 [이데일리 김국배 기자] 메타버스로 사업 영역을 확장하고 있는 메타(전 페이스북)가 인공지능(AI) 연구개발 등을 위해 아마존과 협력을 확대했다. 장기간에 걸쳐 아마존 클라우드를 활용하는 계약을 맺은 것이다. 메타는 2일(현지시간) 미국 라스베이거스에서 아마존웹서비스(AWS)가 개최한 ‘AWS 리인벤트’ 행사에서 AWS를 전략적 클라우드 공급자로 선정했다고 발표했다. 구체적인 계약 기간을 밝히진 않았다. 그간 메타는 자체 데이터센터를 운영하며 일부 업무 시스템에 한해 AWS 클라우드를 활용해왔다. 2012년 인스타그램을 인수한 후에는 인스타그램이 AWS에서 운영해온 대규모 시스템을 자체 데이터센터로 이관시키기까지 했다. 하지만 이번 협력으로 메타의 AWS 클라우드 사용량이 크게 늘어날 것으로 전망된다. 메타의 글로벌 서비스가 전 세계에 걸쳐 25개의 리전(데이터센터)를 보유하고 있는 AWS 클라우드에서 운영될 수 있기 때문이다. 메타기 이번 파트너십의 목적 중 하나로 ‘자체 데이터센터 보완’이라고 밝힌 것도 같은 맥

In [8]:
print(naver_news_title[0])

["메타, 'AI 연구개발' 위해 아마존 클라우드 더 쓴다", '[스타트업] 박지웅과 ‘커리의 3점슛 이론’', '[교육소식]배재대 IPP사업단, 일학습병행사업 IT기업 설명회 등']


In [9]:
print(naver_news_content[0])

["   본문 내용     TV플레이어     // TV플레이어     // flash 오류를 우회하기 위한 함수 추가 function _flash_removeCallback() {}    전략적 클라우드 공급자 선정, 장기 계약 자체 데이터센터 보완, AI 연구개발 가속 서드파티와 협업에도 활용 AWS서 실행되는 딥러닝 프레임워크 '파이토치' 성능 개선 [이데일리 김국배 기자] 메타버스로 사업 영역을 확장하고 있는 메타(전 페이스북)가 인공지능(AI) 연구개발 등을 위해 아마존과 협력을 확대했다. 장기간에 걸쳐 아마존 클라우드를 활용하는 계약을 맺은 것이다. 메타는 2일(현지시간) 미국 라스베이거스에서 아마존웹서비스(AWS)가 개최한 ‘AWS 리인벤트’ 행사에서 AWS를 전략적 클라우드 공급자로 선정했다고 발표했다. 구체적인 계약 기간을 밝히진 않았다. 그간 메타는 자체 데이터센터를 운영하며 일부 업무 시스템에 한해 AWS 클라우드를 활용해왔다. 2012년 인스타그램을 인수한 후에는 인스타그램이 AWS에서 운영해온 대규모 시스템을 자체 데이터센터로 이관시키기까지 했다. 하지만 이번 협력으로 메타의 AWS 클라우드 사용량이 크게 늘어날 것으로 전망된다. 메타의 글로벌 서비스가 전 세계에 걸쳐 25개의 리전(데이터센터)를 보유하고 있는 AWS 클라우드에서 운영될 수 있기 때문이다. 메타기 이번 파트너십의 목적 중 하나로 ‘자체 데이터센터 보완’이라고 밝힌 것도 같은 맥락이다. 메타가 메타버스 사업을 강화하고 있는 만큼 관련 서비스들이 AWS 클라우드에서 운영될 가능성도 배제할 수 없다. 특히 메타는 AWS와 협력을 강화해 AI그룹의 AI 연구 개발에 속도를 낼 계획이다. 메타는 지난 3월 인스타그램 사진 10억장으로 훈련시킨 새로운 AI 알고리즘을 공개하는 등 AI 연구에 박차를 가하고 있다. 또한 AWS 클라우드에서 실행되는 ‘파이토치’ 성능을 개선해 딥러닝 모델을 보다 쉽게 구축·배포할 수 있게 하려 한다. 파이토치는 프로그래밍 언어 ‘파이썬’을 위한 딥러닝

In [10]:
print(len(naver_news_title[0]))
print(len(naver_news_content[0]))

3
3


In [11]:
with open("naver_news_title.pk", "wb") as f:
    pickle.dump(naver_news_title, f)
    
with open("naver_news_content.pk", "wb") as f:
    pickle.dump(naver_news_content, f)